# CIE 5104 Traveler Behavior Analysis: Mid-Term Examination (Take-Home)
## Q1 Box office prediction
### Data collection - TMDB

In [ ]:
%pip install aiohttp
%pip install aiohttp[speedups]

In [247]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import asyncio
from aiohttp import ClientSession
import numpy as np
from sklearn.model_selection import train_test_split
import math
from collections import Counter
from datetime import datetime
from xgboost import XGBRFRegressor
from catboost import CatBoostRegressor
import re
from sklearn.model_selection import GridSearchCV

Sync request too slow, use async package see `TMDB_all.py`

In [76]:
with open('apikey.txt', 'r') as f:
    apikey = f.read()

# get latest movie id
url = 'https://api.themoviedb.org/3/movie/latest?api_key=' + apikey
r = requests.get(url)
data = r.json()
latest_movie_id = data['id']

url = 'https://api.themoviedb.org/3/movie/' + str(157336)+"/credits"+ '?api_key=' + apikey
r = requests.get(url)
if(r.status_code == 200):
    data = r.json()
    print(data['cast'])

[{'adult': False, 'gender': 2, 'id': 10297, 'known_for_department': 'Acting', 'name': 'Matthew McConaughey', 'original_name': 'Matthew McConaughey', 'popularity': 14.947, 'profile_path': '/rDYQP0X5XlDwxOeZdLYMbsIV7F1.jpg', 'cast_id': 9, 'character': "Joseph 'Coop' Cooper", 'credit_id': '52fe4bbf9251416c910e47cb', 'order': 0}, {'adult': False, 'gender': 1, 'id': 1813, 'known_for_department': 'Acting', 'name': 'Anne Hathaway', 'original_name': 'Anne Hathaway', 'popularity': 41.479, 'profile_path': '/tLelKoPNiyJCSEtQTz1FGv4TLGc.jpg', 'cast_id': 169, 'character': 'Dr. Amelia Brand', 'credit_id': '57fe146fc3a368504a00261e', 'order': 1}, {'adult': False, 'gender': 1, 'id': 83002, 'known_for_department': 'Acting', 'name': 'Jessica Chastain', 'original_name': 'Jessica Chastain', 'popularity': 21.5, 'profile_path': '/mR1ELpP0T2Q1ZRSaADPgtrbPGla.jpg', 'cast_id': 17, 'character': "Murphy 'Murph' Cooper", 'credit_id': '52fe4bbf9251416c910e47f1', 'order': 2}, {'adult': False, 'gender': 1, 'id': 956

In [ ]:
# read apikey from file
with open('apikey.txt', 'r') as f:
    apikey = f.read()

# get latest movie id
url = 'https://api.themoviedb.org/3/movie/latest?api_key=' + apikey
r = requests.get(url)
data = r.json()
latest_movie_id = data['id']

# create dataframe to store all movie data
columns = ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
    'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
    'overview', 'popularity', 'poster_path', 'production_companies',
    'production_countries', 'release_date', 'revenue', 'runtime',
    'spoken_languages', 'status', 'tagline', 'title', 'video',
    'vote_average', 'vote_count']
df = pd.DataFrame(columns=columns)

# get movie details
for id in range(latest_movie_id+1):
    url = 'https://api.themoviedb.org/3/movie/' + str(id) + '?api_key=' + apikey
    r = requests.get(url)
    if(r.status_code == 200):
        data = r.json()
        df = df.append(pd.DataFrame.from_dict(data, orient='index').T, ignore_index=True)
df.to_csv('data/movies.csv')

In [ ]:
! python TMDb_all.py

In [97]:
d = pd.read_csv('data/movies.csv')
pred = d.loc[d.id == 453395]
pred

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title
257015,False,"{'id': 618529, 'name': 'Doctor Strange Collect...",170000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",453395,tt9419884,en,Doctor Strange in the Multiverse of Madness,519.885,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2022-05-04,126.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Doctor Strange in the Multiverse of Madness


In [99]:
def get_casts(id):
    url = 'https://api.themoviedb.org/3/movie/' + str(id) + '/credits' + '?api_key=' + apikey
    r = requests.get(url)
    if(r.status_code == 200):
        data = r.json()
        return data['cast']
def get_keywords(id):
    url = 'https://api.themoviedb.org/3/movie/' + str(id) + '/keywords' + '?api_key=' + apikey
    r = requests.get(url)
    if(r.status_code == 200):
        data = r.json()
        return data['keywords']
pred['casts'] = pred['id'].apply(lambda x: get_casts(x))
pred['keywords'] = pred['id'].apply(lambda x: get_keywords(x))

C:\Users\olove\AppData\Local\Temp/ipykernel_19060/2081225285.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['casts'] = pred['id'].apply(lambda x: get_casts(x))
C:\Users\olove\AppData\Local\Temp/ipykernel_19060/2081225285.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['keywords'] = pred['id'].apply(lambda x: get_keywords(x))


In [101]:
pred.to_csv('data/pred.csv')

In [106]:
#df = pd.read_csv('data/movies.csv')
df.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,2,tt0094675,fi,Ariel,...,1988-10-21,0,73.0,"[{'english_name': 'Finnish', 'iso_639_1': 'fi'...",Released,NaN,Ariel,False,6.8,159
1,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,3,tt0092149,fi,Varjoja paratiisissa,...,1986-10-17,0,74.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Shadows in Paradise,False,7.1,161
2,False,/bCXLsO2MJWzYYBJoavGOZN5iqGn.jpg,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",https://www.miramax.com/movie/four-rooms/,5,tt0113101,en,Four Rooms,...,1995-12-09,4257354,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,5.7,2140
3,False,/5aXp2s4l6g5PcMMesIj63mx8hmJ.jpg,NaN,21000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,6,tt0107286,en,Judgment Night,...,1993-10-15,12136938,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,False,6.6,237
4,False,NaN,NaN,42000,"[{'id': 99, 'name': 'Documentary'}]",http://lifeinloops.com,8,tt0825671,en,Life in Loops (A Megacities RMX),...,2006-01-01,0,80.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,A Megacities remix.,Life in Loops (A Megacities RMX),False,7.5,18


| | |
|-|-|
|adult|False|
|backdrop_path| /ndCSoasjIZAMMDIuMxuGnNWu4DU.jpg|
|belongs_to_collection|{'id': 618529, 'name': 'Doctor Strange Collect...|
|budget| 170000000|
|genres| [{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...|
|homepage| https://www.marvel.com/movies/doctor-strange-i...|
|id|453395|
|imdb_id|tt9419884|
|original_language| en|
|original_title| Doctor Strange in the Multiverse of Madness|
|overview| Doctor Strange, with the help of mystical alli...|
|popularity| 519.885|
|poster_path| /wRnbWt44nKjsFPrqSmwYki5vZtF.jpg|
|production_companies| [{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...|
|production_countries| [{'iso_3166_1': 'US', 'name': 'United States o...|
|release_date|2022-05-04|
|revenue|0|
|runtime|126.0|
|spoken_languages| [{'english_name': 'English', 'iso_639_1': 'en'...|
|status| Post Production|
|tagline|Enter a new dimension of Strange.|
|title|Doctor Strange in the Multiverse of Madness|
|video|False|
|vote_average| 0.0|
|vote_count| 0|

remove data without `imdb_id` and `revenue`

In [126]:
df = df.loc[df.imdb_id != np.nan]
df.loc[df.revenue > 0].to_csv('data/movies_with_revenue.csv')

Get opening box office data from IMDB

In [ ]:
! python imdb.py

In [342]:
df = pd.read_csv('data/movies_final.csv')
df = df.loc[df['release_date'].astype('datetime64[ns]')>datetime(1977,1,1)]
df = df.loc[df['original_language'] == 'en']
df.to_csv('data/movies_final.csv', index=False)

In [361]:
import requests
from bs4 import BeautifulSoup
from time import sleep, time

date_format = '%b %d, %Y'
opening_box_office = []
headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3538.102 Safari/537.36 Edge/18.19582"
}
time_s = time()
pro = 0
for id in df['imdb_id']:
    url = 'https://www.imdb.com/title/' + str(id) + '/'
    r = requests.get(url, headers=headers)
    if(r.status_code == 200):
        soup = BeautifulSoup(r.text, 'html5lib')
        box_office = soup.findAll(
            'span', attrs={'class': 'ipc-metadata-list-item__list-content-item'})
        index = -1
        for i, bo in enumerate(box_office):
            try:
                datetime.strptime(bo.text, date_format)
                index = i
                break
            except:
                continue
        if(index == -1):
            opening_box_office.append(np.nan)
            print(f'{id} not found')
            continue
        try:
            box_office = int(box_office[index-1].text[1:].replace(',', ''))
            opening_box_office.append(box_office)
        except:
            opening_box_office.append(np.nan)
            print(f'{id} last part')
    elif(r.status_code == 503):
        print(f'{id} 503')
    pro += 1
    if(pro % 800 == 0):
        print(pro)
print(f'{time()-time_s} seconds')

tt0089015 not found
tt0285492 not found
tt0242527 not found
tt0102494 not found
tt0110729 not found
tt0087884 not found
tt0076752 not found
tt0108122 not found
tt0075686 not found
tt0083907 not found
tt0080339 not found
tt0077402 not found
tt0077651 not found
tt0074486 not found
tt0078966 not found
tt0082340 not found
tt0443456 not found
tt1344607 not found
tt1344607 last part
tt0445946 not found
tt1012729 not found
tt0080678 not found
tt0416496 not found
tt1053859 not found
tt1028528 not found
tt0211792 not found
tt0211792 last part
tt0110044 not found
tt0113063 not found
tt0113063 last part
800
tt0307596 not found
tt0408281 not found
tt0379561 not found
tt0379561 last part
tt0429153 not found
tt0077278 not found
tt0077413 not found
tt0092944 not found
tt0445935 not found
tt0289889 not found
tt0402910 not found
tt0100946 not found
tt0120604 not found
tt0080453 not found
tt0075784 not found
tt0081163 not found
tt0078163 not found
tt0091886 not found
tt0790665 not found
tt0078492 not fo

In [481]:
df.loc[df['imdb_id'].isin(d[1].values), 'opening_box_office'] = np.nan

In [485]:
df.to_csv('movies_0.csv', index=False)

In [487]:
df = pd.read_csv('data/movies_0_box.csv').dropna(subset=['opening_box_office'])
df

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title,opening_box_office,casts,keywords
0,False,NaN,4000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",5.0,tt0113101,en,Four Rooms,18.437,"[{'id': 14, 'logo_path': '/m6AHu84oZQxvq7n1rsv...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-09,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Four Rooms,427733.0,"[{'adult': False, 'gender': 2, 'id': 3129, 'kn...","[{'id': 612, 'name': 'hotel'}, {'id': 613, 'na..."
1,False,NaN,21000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",6.0,tt0107286,en,Judgment Night,10.566,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",1993-10-15,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Judgment Night,4088955.0,"[{'adult': False, 'gender': 2, 'id': 2880, 'kn...","[{'id': 520, 'name': 'chicago, illinois'}, {'i..."
2,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11.0,tt0076759,en,Star Wars,89.361,"[{'id': 1, 'logo_path': '/o86DbpburjxrqAzEDhXZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",1977-05-25,121.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Star Wars,1554475.0,"[{'adult': False, 'gender': 2, 'id': 2, 'known...","[{'id': 803, 'name': 'android'}, {'id': 4270, ..."
3,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",12.0,tt0266543,en,Finding Nemo,132.131,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-05-30,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Finding Nemo,70251710.0,"[{'adult': False, 'gender': 2, 'id': 13, 'know...","[{'id': 970, 'name': 'parent child relationshi..."
4,False,NaN,55000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",13.0,tt0109830,en,Forrest Gump,67.124,"[{'id': 4, 'logo_path': '/fycMZt242LVjagMByZOL...","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-07-06,142.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Forrest Gump,24450602.0,"[{'adult': False, 'gender': 2, 'id': 31, 'know...","[{'id': 422, 'name': 'vietnam veteran'}, {'id'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8813,False,NaN,43000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",823625.0,tt14060094,en,Blacklight,2217.388,"[{'id': 33768, 'logo_path': '/f8TDXTNoj4G4eRCK...","[{'iso_3166_1': 'AU', 'name': 'Australia'}, {'...",2022-02-10,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Blacklight,3502475.0,"[{'adult': False, 'gender': 2, 'id': 3896, 'kn...",[]
8822,False,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",837548.0,tt14640242,en,Together,10.715,"[{'id': 3324, 'logo_path': '/dqT3yOTlfJRmtvk52...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",2021-08-27,87.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Together,100066.0,"[{'adult': False, 'gender': 2, 'id': 5530, 'kn...","[{'id': 6038, 'name': 'marriage'}, {'id': 1865..."
8863,False,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",895744.0,tt15374070,en,Studio 666,28.001,"[{'id': 48176, 'logo_path': None, 'name': 'Ros...","[{'iso_3166_1': 'US', 'name': 'United States o...",2022-02-24,106.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Studio 666,1540543.0,"[{'adult': False, 'gender': 2, 'id': 533061, '...","[{'id': 11181, 'name': 'decapitation'}, {'id':..."
8898,False,NaN,0.0,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 18,...",905068.0,tt15772866,en,Christmas with The Chosen: The Messengers,3.964,"[{'id': 165435, 'logo_path': '/cV8GBoy85v6aWxN...","[{'iso_3166_1': 'US', 'name': 'United States o...",2021-12-01,110.0,[],Christmas with The Chosen: The Messengers,4245000.0,"[{'adult': False, 'gender': 0, 'id': 3328694, ...",

In [497]:
df['keywords'].fillna('[]', inplace=True)

In [232]:
pred = pd.read_csv('data/pred.csv', index_col=0)
pred

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title,casts,keywords
257015,False,"{'id': 618529, 'name': 'Doctor Strange Collect...",170000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",453395,tt9419884,en,Doctor Strange in the Multiverse of Madness,519.885,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2022-05-04,126.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Doctor Strange in the Multiverse of Madness,"[{'adult': False, 'gender': 2, 'id': 71580, 'k...","[{'id': 2343, 'name': 'magic'}, {'id': 9717, '..."


In [498]:
list_of_genres = sum(list(df['genres'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_genres = [m[0] for m in Counter(list_of_genres).most_common(15)]

list_of_production_companies = sum(list(df['production_companies'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_production_companies = [m[0] for m in Counter(list_of_production_companies).most_common(30)]

list_of_production_countries = sum(list(df['production_countries'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_production_countries = [m[0] for m in Counter(list_of_production_countries).most_common(25)]

list_of_spoken_languages = sum(list(df['spoken_languages'].apply(lambda x: [i['english_name'] for i in eval(x)])), [])
top_spoken_languages = [m[0] for m in Counter(list_of_spoken_languages).most_common(15)]

list_of_keywords = sum(list(df['keywords'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_keywords = [m[0] for m in Counter(list_of_keywords).most_common(30)]

list_of_casts_names = sum(list(df['casts'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_casts_names = [m[0] for m in Counter(list_of_casts_names).most_common(15)]

list_of_casts_characters = sum(list(df['casts'].apply(lambda x: [i['character'] for i in eval(x)])), [])
top_casts_characters = [m[0] for m in Counter(list_of_casts_characters).most_common(15)]

In [233]:
def collection_preprocessing(x):
    x = str(x)
    if x == 'nan':
        return -1
    else:
        return int(eval(x)['id'])
        
def get_days_diff(x):
    date_format = "%Y-%m-%d"
    date_base = datetime.strptime('2022-05-04', date_format)
    date_target = datetime.strptime(x, date_format)
    return (date_target - date_base).days
    

def pre_processing(X):
    X.drop(['id', 'imdb_id'], axis=1, inplace=True)
    X['genres'] = X['genres'].apply(lambda x: eval(x))
    X['production_companies'] = X['production_companies'].apply(lambda x: eval(x))
    X['production_countries'] = X['production_countries'].apply(lambda x: eval(x))
    X['spoken_languages'] = X['spoken_languages'].apply(lambda x: eval(x))
    X['casts'] = X['casts'].apply(lambda x: eval(x))
    X['keywords'] = X['keywords'].apply(lambda x: eval(x))

    X['collection_id'] = X['belongs_to_collection'].apply(collection_preprocessing)
    X['has_collection'] = X['collection_id'].apply(lambda x: 1 if x != '' else 0)
    X = X.drop(['belongs_to_collection'], axis=1)

    X['genres_list'] = X['genres'].apply(lambda x: [i['name'] for i in x])
    X['num_genres'] = X['genres_list'].apply(len)
    for g in top_genres:
        X['genre_' + g] = X['genres_list'].apply(lambda x: 1 if g in x else 0)
    X = X.drop(['genres', 'genres_list'], axis=1)
    
    X['production_companies_list'] = X['production_companies'].apply(lambda x: [i['name'] for i in x])
    X['num_production_companies'] = X['production_companies_list'].apply(len)
    for c in top_production_companies:
        X['production_company_' + c] = X['production_companies_list'].apply(lambda x: 1 if c in x else 0)
    X = X.drop(['production_companies', 'production_companies_list'], axis=1)

    X['production_countries_list'] = X['production_countries'].apply(lambda x: [i['name'] for i in x])
    X['num_production_countries'] = X['production_countries_list'].apply(len)
    for c in top_production_countries:
        X['production_country_' + c] = X['production_countries_list'].apply(lambda x: 1 if c in x else 0)
    X = X.drop(['production_countries', 'production_countries_list'], axis=1)

    X['spoken_languages_list'] = X['spoken_languages'].apply(lambda x: [i['english_name'] for i in x])
    X['num_spoken_languages'] = X['spoken_languages_list'].apply(len)
    for l in top_spoken_languages:
        X['spoken_language_' + l] = X['spoken_languages_list'].apply(lambda x: 1 if l in x else 0)
    X = X.drop(['spoken_languages', 'spoken_languages_list'], axis=1)

    X['keywords_list'] = X['keywords'].apply(lambda x: [i['name'] for i in x])
    X['num_keywords'] = X['keywords_list'].apply(len)
    for k in top_keywords:
        X['keyword_' + k] = X['keywords_list'].apply(lambda x: 1 if k in x else 0)
    X = X.drop(['keywords', 'keywords_list'], axis=1)
    
    X['casts_list'] = X['casts'].apply(lambda x: [i['name'] for i in x])
    X['num_casts'] = X['casts_list'].apply(len)
    for c in top_casts_names:
        X['cast_' + c] = X['casts_list'].apply(lambda x: 1 if c in x else 0)
    for c in top_casts_characters:
        X['cast_character_' + c] = X['casts_list'].apply(lambda x: 1 if c in x else 0)
    X['casts_gender_0'] = X['casts'].apply(lambda x: sum([1 for i in x if ['gender'] == 0]))
    X['casts_gender_1'] = X['casts'].apply(lambda x: sum([1 for i in x if ['gender'] == 1]))
    X['casts_gender_2'] = X['casts'].apply(lambda x: sum([1 for i in x if ['gender'] == 2]))    X = X.drop(['casts', 'casts_list'], axis=1)

    X['log_budget'] = np.log1p(X['budget'])

    X['days_before'] = X['release_date'].apply(get_days_diff)
    X = X.drop(['release_date'], axis=1)

    X = X.drop(['original_language', 'original_title', 'title'], axis=1)
    X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    return X

In [518]:
#http://www.boxofficereport.com/trailerviews/owyoutuberatios.html
df_YT = pd.read_clipboard(sep = '\t')
df_YT.to_csv('data/YT.csv', index=False)

In [523]:
df_YT.columns = ['title', 'release_date', 'link', 'opneing_weekend', 'trailer_views', 'ow']
df_YT = df_YT[['title', 'trailer_views']]
df_YT.head()

,title,trailer_views
0,American Underdog (Lionsgate / Kingdom Story),0.361 M
1,Sing 2 (Universal) #,27.934 M
2,Licorice Pizza (United Artists Releasing / MGM),6.839 M
3,A Journal for Jordan (Sony / Columbia),9.366 M
4,The Matrix Resurrections (Warner Bros.) #,56.034 M


In [530]:
def get_title(text):
    if text.endswith('#'):
        text = text[:-2]
    index_left = text.rfind('(')
    return text[:index_left-1]
df_YT['title'] = df_YT['title'].apply(get_title)
def get_views(text):
    return float(text[:-2]) * 1000000
df_YT['trailer_views'] = df_YT['trailer_views'].apply(get_views)

C:\Users\olove\AppData\Local\Temp/ipykernel_19060/2748823482.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_YT['title'] = df_YT['title'].apply(get_title)


In [531]:
df_YT

,title,trailer_views
0,American Underdog,361000.0
1,Sing 2,27934000.0
2,Licorice Pizza,6839000.0
3,A Journal for Jordan,9366000.0
4,The Matrix Resurrections,56034000.0
...,...,...
583,Live By Night,7067000.0
584,Silence,5020000.0
585,Hidden Figures,7739000.0
586,Underworld: Blood Wars,16795000.0


In [537]:
df_YT.title

0             American Underdog
1                        Sing 2
2                Licorice Pizza
3          A Journal for Jordan
4      The Matrix Resurrections
                 ...           
583               Live By Night
584                     Silence
585              Hidden Figures
586      Underworld: Blood Wars
587             A Monster Calls
Name: title, Length: 588, dtype: object

In [550]:
df_views = df.merge(df_YT, on='title', how='left')

In [552]:
df_views.to_csv('data/movies_views.csv', index=False)

In [555]:
pd.read_clipboard(sep=',').nlargest(10, 'value')

,index,feature,value
0,121,keyword_basedoncomic,0.079653
1,1,budget,0.071764
2,129,cast_SamuelLJackson,0.055552
3,166,month_4,0.048387
4,161,log_budget,0.039923
5,142,cast_StanleyTucci,0.037578
6,4,trailer_views,0.036527
7,30,production_company_WaltDisneyPictures,0.033826
8,173,month_11,0.030960
9,54,num_production_countries,0.029923


In [549]:
df_3h_views

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title,opening_box_office,casts,keywords,trailer_views
0,False,"{'id': 86311, 'name': 'The Avengers Collection...",356000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...",299534.0,tt4154796,en,Avengers: Endgame,315.107,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2019-04-24,181.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Avengers: Endgame,357115007.0,"[{'adult': False, 'gender': 2, 'id': 3223, 'kn...","[{'id': 3801, 'name': 'space travel'}, {'id': ...",233372000.0
1,False,"{'id': 531241, 'name': 'Spider-Man (Avengers) ...",200000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",634649.0,tt10872600,en,Spider-Man: No Way Home,7358.500,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2021-12-15,148.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Spider-Man: No Way Home,260138569.0,"[{'adult': False, 'gender': 2, 'id': 1136406, ...","[{'id': 242, 'name': 'new york city'}, {'id': ...",196831000.0
2,False,"{'id': 86311, 'name': 'The Avengers Collection...",300000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",299536.0,tt4154756,en,Avengers: Infinity War,454.608,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2018-04-25,149.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Avengers: Infinity War,257698183.0,"[{'adult': False, 'gender': 2, 'id': 3223, 'kn...","[{'id': 2343, 'name': 'magic'}, {'id': 2858, '...",243509000.0
3,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",245000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",140607.0,tt2488496,en,Star Wars: The Force Awakens,88.269,"[{'id': 1, 'logo_path': '/o86DbpburjxrqAzEDhXZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2015-12-15,136.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Star Wars: The Force Awakens,247966675.0,"[{'adult': False, 'gender': 2, 'id': 3, 'known...","[{'id': 803, 'name': 'android'}, {'id': 1612, ...",NaN
4,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",200000000.0,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",181808.0,tt2527336,en,Star Wars: The Last Jedi,81.640,"[{'id': 1, 'logo_path': '/o86DbpburjxrqAzEDhXZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2017-12-13,152.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Star Wars: The Last Jedi,220009584.0,"[{'adult': False, 'gender': 2, 'id': 2, 'known...","[{'id': 2321, 'name': 'bunker'}, {'id': 2902, ...",92221000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,False,"{'id': 51509, 'name': 'Meet the Parents Collec...",80000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",693.0,tt0290002,en,Meet the Fockers,27.373,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'US', 'name': 'United States o...",2004-12-22,115.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Meet the Fockers,46120980.0,"[{'adult': False, 'gender': 2, 'id': 7399, 'kn...","[{'id': 428, 'name': 'nurse'}, {'id': 591, 'na...",NaN
296,False,NaN,144000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 14, 'nam...",43074.0,tt1289401,en,Ghostbusters,83.048,"[{'id': 84042, 'logo_path': None, 'name': 'Gho...","[{'iso_3166_1': 'US', 'name': 'United States o...",2016-07-14,117.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Ghostbusters,46018755.0,"[{'adult': False, 'gender': 1, 'id': 55536, 'k...","[{'id': 5248, 'name': 'female friendship'}, {'...",NaN
297,False,NaN,130000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 28, '...",38055.0,tt1001526,en,Megamind,82.355,"[{'id': 521, 'logo_path': '/kP7t6RwGz2AvvTkvnI...","[{'iso_3166_1': 'US', 'name': 'United States o...",2010-10-28,96.0,"[{'english_name': 'English', 'iso_639_1': 'en'.

In [514]:
train_X.columns

Index(['adult', 'budget', 'popularity', 'runtime', 'collection_id',
       'has_collection', 'num_genres', 'genre_Drama', 'genre_Comedy',
       'genre_Action',
       ...
       'cast_character_Bartender', 'cast_character_uncredited',
       'cast_character_Waitress', 'cast_character_Selfarchivefootage',
       'cast_character_Waiter', 'casts_gender_0', 'casts_gender_1',
       'casts_gender_2', 'log_budget', 'days_before'],
      dtype='object', length=162)

In [234]:
train_X, test_X, train_y, test_y = train_test_split(
    df.drop(['opening_box_office'], axis=1), df['opening_box_office'],
    test_size=0.2, random_state=7414)

train_X = pre_processing(train_X)
test_X = pre_processing(test_X)
pred = pre_processing(pred)

In [248]:
XGBR = XGBRFRegressor(seed = 7414)

parameters = {
'max_depth': [5, 10, 15, 20, 25],
'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
'n_estimators': [500, 1000, 2000, 3000, 5000],
'min_child_weight': [0, 2, 5, 10, 20],
'max_delta_step': [0, 0.2, 0.6, 1, 2],
'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]
}

gsearch = GridSearchCV(estimator = XGBR, param_grid=parameters, scoring='neg_mean_squared_error', cv=5, n_jobs=4, verbose=1)
gsearch.fit(train_X, train_y)
best_parameters, score, _ = max(gsearch.grid_scores_, key=lambda x: x[1])
print(best_parameters, score)

Fitting 5 folds for each of 9765625 candidates, totalling 48828125 fits


KeyboardInterrupt: 